# gridfinder
Run through the full gridfinder model from data input to final guess for Burundi.
Note that the 'truth' data used for the grid here is very bad, so the accuracy results don't mean much.

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import display, Markdown

import numpy as np
import rasterio
import geopandas as gpd
import folium

import gridfinder as gf
from gridfinder import save_raster

## Set folders and parameters

In [ ]:
folder_inputs = Path('test_data')
folder_ntl_in = folder_inputs / 'ntl'
aoi_in = folder_inputs / 'gadm.gpkg'
roads_in = folder_inputs / 'roads.gpkg'
pop_in = folder_inputs / 'pop.tif'
grid_truth = folder_inputs / 'grid.gpkg'

folder_out = Path('test_output')
folder_ntl_out = folder_out / 'ntl_clipped'
raster_merged_out = folder_out / 'ntl_merged.tif'
targets_out = folder_out / 'targets.tif'
targets_clean_out = folder_out / 'targets_clean.tif'
roads_out = folder_out / 'roads.tif'

dist_out = folder_out / 'dist.tif'
guess_out = folder_out / 'guess.tif'
guess_skeletonized_out = folder_out / 'guess_skel.tif'
guess_nulled = folder_out / 'guess_nulled.tif'
guess_vec_out = folder_out / 'guess.gpkg'
animate_out = folder_out / 'animated'

In [ ]:
percentile = 70      # percentile value to use when merging monthly NTL rasters
ntl_threshold = 0.1  # threshold when converting filtered NTL to binary (probably shouldn't change)
upsample_by = 2      # factor by which to upsample before processing roads (both dimensions are scaled by this)
cutoff = 0.0         # cutoff to apply to output dist raster, values below this are considered grid

## Clip  and merge monthly rasters

In [ ]:
gf.clip_rasters(folder_ntl_in, folder_ntl_out, aoi_in)

In [ ]:
raster_merged, affine = gf.merge_rasters(folder_ntl_out, percentile=percentile)
save_raster(raster_merged_out, raster_merged, affine)
print('Merged')
plt.imshow(raster_merged, vmin=0, vmax=1)

## Create filter

In [ ]:
ntl_filter = gf.create_filter()

X = np.fromfunction(lambda i, j: i, ntl_filter.shape)
Y = np.fromfunction(lambda i, j: j, ntl_filter.shape)

fig = plt.figure()
sns.set()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(X, Y, ntl_filter, cmap=cm.coolwarm, linewidth=0, antialiased=False)

## Clip, filter and resample NTL

In [ ]:
ntl_thresh, affine = gf.prepare_ntl(raster_merged_out,
                                    aoi_in,
                                    ntl_filter=ntl_filter,
                                    threshold=ntl_threshold,
                                    upsample_by=upsample_by)
save_raster(targets_out, ntl_thresh, affine)
print('Targets prepared')
plt.imshow(ntl_thresh, cmap='viridis')

## Remove target areas with no underlying population

In [ ]:
targets_clean = gf.drop_zero_pop(targets_out, pop_in, aoi_in)
save_raster(targets_clean_out, targets_clean, affine)
print('Removed zero pop')
plt.imshow(ntl_thresh, cmap='viridis')

## Roads: assign values, clip and rasterize

In [ ]:
roads_raster, affine = gf.prepare_roads(roads_in,
                                        aoi_in,
                                        targets_out)
save_raster(roads_out, roads_raster, affine, nodata=-1)
print('Costs prepared')
plt.imshow(roads_raster, cmap='viridis', vmin=0, vmax=1)

## Get targets and costs and run algorithm

In [ ]:
targets, costs, start, affine = gf.get_targets_costs(targets_clean_out, roads_out)
est_mem = gf.estimate_mem_use(targets, costs)
print(f'Estimated memory usage: {est_mem:.2f} GB')

In [ ]:
dist = gf.optimise(targets, costs, start,
                   jupyter=True,
                   animate=True,
                   affine=affine,
                   animate_path=animate_out)
save_raster(dist_out, dist, affine)
plt.imshow(dist)

## Filter dist results to grid guess

In [ ]:
guess, affine = gf.threshold(dist_out, cutoff=cutoff)
save_raster(guess_out, guess, affine)
print('Got guess')
plt.imshow(guess, cmap='viridis')

## Check results

In [ ]:
true_pos, false_neg = gf.accuracy(grid_truth, guess_out, aoi_in)
print(f'Points identified as grid that are grid: {100*true_pos:.0f}%')
print(f'Actual grid that was missed: {100*false_neg:.0f}%')

## Skeletonize

In [ ]:
guess_skel, affine = gf.thin(guess_out)
save_raster(guess_skeletonized_out, guess_skel, affine)
print('Skeletonized')
plt.imshow(guess_skel)

## Convert to geometry

In [ ]:
guess_gdf = gf.raster_to_lines(guess_skeletonized_out)
guess_gdf.to_file(guess_vec_out, driver='GPKG')
print('Converted to geom')

In [ ]:
minx, miny, maxx, maxy = list(guess_gdf.bounds.iloc[0])
bounds = ((miny, minx), (maxy, maxx))

m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
folium.GeoJson(guess_gdf).add_to(m)
m